In [1]:
# read in libraries
import pandas as pd
import numpy as np
import re
import os
import json
import sys
import datetime as dt
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tner import TransformersNER

# connect to sql
from mysql.connector import (connection)
from mysql.connector import Error

# load functions
with open('project_config.json','r') as fp: 
    project_config = json.load(fp)

module_path = os.path.join(project_config['project_module_relative_path'])
sys.path.append(module_path)

from ner.ner_processing import *




# SQL connection ------------------------------------------------------------------

# fetch values from environment variables and set the target database
hostname = os.environ['MYSQL_HOST']
username = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']
dbname = 'found'

# establish connection to db1 database in your mysql service
cnx = connection.MySQLConnection(user=username,
                                 password=password,
                                 host=hostname,
                                 database=dbname)

# connect to sql
from mysql.connector import (connection)
from mysql.connector import Error

# fetch values from environment variables and set the target database
hostname = os.environ['MYSQL_HOST']
username = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']
dbname = 'found'

# establish connection to db1 database in your mysql service
cnx = connection.MySQLConnection(user=username,
                                 password=password,
                                 host=hostname,
                                 database=dbname)


2023-03-24 17:42:21.447842: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/oracle/instantclient_12_1:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server:
2023-03-24 17:42:21.447883: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# query database ---------------------------------------------------------------------
query =("SELECT * FROM landmine_tweets WHERE date > CURDATE() - INTERVAL 2 day")

#write to df and remove duplicates
df = pd.read_sql_query(query, cnx)
df = df.drop_duplicates(subset=['tweet', 'handle'], keep="first")
df.columns = df.columns.str.lower()

In [3]:
# detect language --------------------------------------------------------------------
nlp = en_core_web_sm.load()


In [4]:
# load spacy nlp model
nlp_model = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp_model.add_pipe('language_detector', last=True)

# create a language column on dataframe
df["language"]= df["tweet"].apply(get_language)


In [5]:
# separate dataframe by eng and non eng tweets --------------------------------------------
# create dataframe for non english tweets
non_eng = df[df["language"] != "en"]

# write to csv
#non_eng.to_csv("landmine_tweets_non_eng_processed.csv", index = False)

# create dataframe for english tweets
en = df[df["language"]=="en"]

# clean text on english tweets
en =  clean_tweets(en, 'tweet')

# write to csv
#en.to_csv("landmine_tweets_eng_processed.csv", index = False)

# write back to main tweeets file
main = pd.read_csv("main.csv")
main = pd.concat([non_eng, en, main])
#main.to_csv("main.csv", index = False)


src/ner/ner_processing.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[tweets] = df[tweets].str.lower()
src/ner/ner_processing.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[tweets] = df[tweets].map(lambda x: re.sub('http[s]?:\/\/[^\s]*', ' ', x))
src/ner/ner_processing.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [30]:
if non_eng.shape[0]!=0:

    tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
    model = AutoModelForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)

    # apply ner model for non eng tweets
    non_eng["ner"] = non_eng["tweet"].apply(nlp)

    # list comprehension to combine all ner records
    ner_records = []
    for row_dict in non_eng.to_dict(orient="records"):
        for ner_tag_dict in nlp(row_dict["tweet"]):  # I Don't recall exact ner model function call
            ner_records.append({**row_dict, **ner_tag_dict})

    # create ner tag df
    ner_df = create_ner_tag_df_multi(ner_records, "Davlan/bert-base-multilingual-cased-ner-hrl")

    # removing all English characters from ner tag df because the multilingual model isn't great for English text

    # detect language of ner tags
    ner_df["language"]= ner_df["tagged_text"].apply(isEnglish)

    # drop English rows
    ner_df= ner_df[ner_df["language"] == False].drop(columns="language", axis=True)

    # cleaning text
    ner_df['tagged_text'] = ner_df['tagged_text'].fillna(" ")
    ner_df['tagged_text'] = ner_df["tagged_text"].str.replace("##", "")

    # recombine NER tags
    ner_df_combined = combine_ner_tags(ner_df, "")

    # write to csv
    #ner_df.to_csv("landmine_tweets_ner_results_non_eng_clean.csv", index = False)

else:
    print("No non-English tweets today")


No non-english tags


In [12]:
ner_df = create_ner_tag_df_multi(ner_records, "Davlan/bert-base-multilingual-cased-ner-hrl")

In [14]:
ner_df["language"]= ner_df["tagged_text"].apply(isEnglish)

In [15]:
ner_df["language"].value_counts()

True    9
Name: language, dtype: int64

In [27]:
# function to combine ner tags
def combine_ner_tags(df, spacer):
    """
    Function to recombine NER tags from B- to I-
    df = dataframe
    spacer = what kind of spacing to use to combine text. Use "" for non english and " " for english
    """
    combined_tokens = []
    df = df.reset_index()
    if df.shape[0] !=0:
        if df.iloc[0]["ner_tag"].startswith("I-"):
            df=df.iloc[1:,]
        df = df.reset_index()
        cur_tokens = [df.loc[0, "tagged_text"]]
        for i in range(1, len(df)):
            if df.loc[i, "ner_tag"].startswith("B-"):
                combined_tokens.append(cur_tokens)
                cur_tokens = [df.loc[i, "tagged_text"]]
            elif df.loc[i, "ner_tag"].startswith("I-"):
                 cur_tokens.append(df.loc[i, "tagged_text"])
        b_tags = df[df["ner_tag"].str.startswith("B-")]
        combined_tokens.append([b_tags.tail(1)["tagged_text"].values[0]])
        # update the combined text join based on which language. no space for non_eng
        combined_text = [spacer.join(i) for i in combined_tokens]
        b_tags["tagged_text"] = combined_text
        return b_tags
    else:
        print("No non-english tags")
    


In [28]:
ner_df_combined = combine_ner_tags(ner_df, "")

/tmp/ipykernel_553/1065089801.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_tags["tagged_text"] = combined_text


In [29]:
ner_df_combined

,level_0,index,document_id,sentence_number,sentence_text,tagged_text,ner_tag,tagged_text_loci,ner_model,probability,language
0,0,0,1638752507802554368,1,Sebagai warga RoG yang mayoritas Muhammadiyah ...,R##o##G,B-ORG,3,Davlan/bert-base-multilingual-cased-ner-hrl,0.999267,True
3,3,3,1638752507802554368,1,Sebagai warga RoG yang mayoritas Muhammadiyah ...,Muhammad##iyah,B-ORG,9,Davlan/bert-base-multilingual-cased-ner-hrl,0.999034,True
5,5,5,1638752507802554368,1,Sebagai warga RoG yang mayoritas Muhammadiyah ...,Muhammad##iyah,B-ORG,19,Davlan/bert-base-multilingual-cased-ner-hrl,0.999407,True
7,7,7,1638752507802554368,1,Sebagai warga RoG yang mayoritas Muhammadiyah ...,Howard,B-PER,32,Davlan/bert-base-multilingual-cased-ner-hrl,0.999777,True


In [30]:
ner_df.shape

(0, 8)

In [29]:
ner_df.iloc[0]

IndexError: single positional indexer is out-of-bounds